In [1]:
print('its working')

its working


In [3]:
%cd /content/LLM-training

/content/LLM-training


In [2]:
!pip -q install -U "transformers>=4.41" "datasets>=2.19" "accelerate>=0.30" "trl>=0.9" "peft>=0.11" bitsandbytes
print('success')

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 121.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 44.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.9/530.9 kB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 13.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 14.9 MB/s eta 0:00:00
success


In [4]:
!pip install --upgrade --force-reinstall pyarrow

  Using cached pyarrow-23.0.0-cp312-cp312-manylinux_2_28_x86_64.whl.metadata (3.0 kB)
Using cached pyarrow-23.0.0-cp312-cp312-manylinux_2_28_x86_64.whl (47.6 MB)
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 23.0.0
    Uninstalling pyarrow-23.0.0:
      Successfully uninstalled pyarrow-23.0.0


In [5]:
import pandas as pd
from datasets import Dataset, DatasetDict
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

In [6]:
CSV_PATH = "/content/LLM-training/dataset/edtech_sft_dataset_100_with_company_info.csv"
df = pd.read_csv(CSV_PATH)

# Preprocessing
df["input"]  = df["input"].astype(str).str.replace("â€™", "’").str.replace("â€“", "–").str.replace("â€œ", "“").str.replace("â€˜", "‘").str.replace("â€", "”")
df["output"] = df["output"].astype(str).str.replace("â€™", "’").str.replace("â€“", "–").str.replace("â€œ", "“").str.replace("â€˜", "‘").str.replace("â€", "”")

# Since the dataset is small, we used the entire dataset for training to improve the model’s performance.
full_dataset = Dataset.from_pandas(df)

# Create 20% validation set FROM the same dataset
eval_dataset = full_dataset.shuffle(seed=42).select(
    range(int(0.2 * len(full_dataset)))
)

# Train uses ALL data
ds = DatasetDict({
    "train": full_dataset,   # 100%
    "test": eval_dataset     # 20% subset of the same data
})

print("Total samples:", len(full_dataset))
print("Train samples (100%):", len(ds["train"]))
print("Validation samples (20%):", len(ds["test"]))


Total samples: 105
Train samples (100%): 105
Validation samples (20%): 21


In [7]:
SYSTEM = "You are a helpful customer support assistant for an ed-tech platform. Answer clearly and politely."

def format_example(ex):
    # Keep it simple + consistent (works for any base causal LM)
    return {
        "text": f"{SYSTEM}\n\nUser: {ex['input']}\nAssistant: {ex['output']}"
    }

ds = ds.map(format_example, remove_columns=ds["train"].column_names)
ds


Map:   0%|          | 0/105 [00:00<?, ? examples/s]

Map:   0%|          | 0/21 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text'],
        num_rows: 105
    })
    test: Dataset({
        features: ['text'],
        num_rows: 21
    })
})

In [8]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained(model_name)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/679 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors:   0%|          | 0.00/3.55G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/181 [00:00<?, ?B/s]

In [9]:
from peft import LoraConfig

peft_config = LoraConfig(
    r=6,                    # small rank (dataset is small)
    lora_alpha=8,           # ~2x rank
    lora_dropout=0.05,
    bias="none",
    target_modules="all-linear",
    task_type="CAUSAL_LM",
)


In [10]:
from trl import SFTConfig

args = SFTConfig(
    output_dir="./lora_adapter_out",
    max_steps=300,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    learning_rate=2e-4,
    logging_steps=10,
    save_steps=100,
    eval_strategy="steps",
    eval_steps=50,
    fp16=True,
    bf16=False,
)


In [11]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=ds["train"],
    eval_dataset=ds["test"],
    peft_config=peft_config,
    processing_class=tokenizer,
)

trainer.train()


Adding EOS to train dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/105 [00:00<?, ? examples/s]

Adding EOS to eval dataset:   0%|          | 0/21 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/21 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/21 [00:00<?, ? examples/s]

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': 151646, 'pad_token_id': 151643}.


Step,Training Loss,Validation Loss
50,0.956315,0.761045
100,0.160641,0.110268
150,0.065302,0.065713
200,0.067799,0.062272
250,0.060125,0.061200
300,0.056742,0.059604


TrainOutput(global_step=300, training_loss=0.3878033785025279, metrics={'train_runtime': 1133.5853, 'train_samples_per_second': 2.117, 'train_steps_per_second': 0.265, 'total_flos': 1580413954243584.0, 'train_loss': 0.3878033785025279})

In [12]:
trainer.save_model("./lora_adapter_out")
tokenizer.save_pretrained("./lora_adapter_out")

('./lora_adapter_out/tokenizer_config.json',
 './lora_adapter_out/chat_template.jinja',
 './lora_adapter_out/tokenizer.json')

In [13]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

base_model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"

tokenizer = AutoTokenizer.from_pretrained("./lora_adapter_out")

base_model = AutoModelForCausalLM.from_pretrained(
    base_model_name,
    torch_dtype=torch.float16,
    device_map="auto"
)


Loading weights:   0%|          | 0/339 [00:00<?, ?it/s]

In [14]:
model = PeftModel.from_pretrained(
    base_model,
    "./lora_adapter_out",
    torch_dtype=torch.float16
)

model.eval()


PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): Qwen2ForCausalLM(
      (model): Qwen2Model(
        (embed_tokens): Embedding(151936, 1536)
        (layers): ModuleList(
          (0-27): 28 x Qwen2DecoderLayer(
            (self_attn): Qwen2Attention(
              (q_proj): lora.Linear(
                (base_layer): Linear(in_features=1536, out_features=1536, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.05, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1536, out_features=6, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=6, out_features=1536, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj): lora.Linear(
 

In [15]:

SYSTEM = "You are a helpful customer support assistant for an ed-tech platform. Answer clearly and politely."

print("Chat started. Type 'exit' to stop.\n")

while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    prompt = f"""{SYSTEM}

User: {user_input}
Assistant:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=120,
            temperature=0.3,
            top_p=0.9,
            do_sample=True
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print("\nAssistant:", response.split("Assistant:")[-1].strip(), "\n")

Chat started. Type 'exit' to stop.

User: Hi


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: Hi too. What’s up? 

User: who are you


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: I’m an instructor for SkillSpring Academy. I help you get started with our platform and prepare for a certificate. 

User: who is the ceo of SkillSpring Academy


...


Assistant: The CEO of SkillSpring Academy is Injamul Haque. 

User: Where is SkillSpring Academy located?


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: SkillSpring Academy is based in Dhaka, Bangladesh and serves learners worldwide online. 

User: Thanks


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: You’re welcome! 

User: exit


In [16]:

SYSTEM = "You are a helpful customer support assistant for an ed-tech platform. Answer clearly and politely."

print("Chat started. Type 'exit' to stop.\n")

while True:
    user_input = input("User: ")
    if user_input.lower() in ["exit", "quit"]:
        break

    prompt = f"""{SYSTEM}

User: {user_input}
Assistant:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=120,
            temperature=0.3,
            top_p=0.9,
            do_sample=True
        )

    response = tokenizer.decode(output[0], skip_special_tokens=True)
    print("\nAssistant:", response.split("Assistant:")[-1].strip(), "\n")

Chat started. Type 'exit' to stop.

User: What payment methods do you accept for 'Python for Data Science'?


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: We typically accept card payments (Visa/Mastercard) and supported local payment options (if available in your region). At checkout you’ll see the exact methods for your location. 

User: I forgot my password and can’t log in.


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: Use the ‘Forgot password’ link on the login page to reset your password. If you don’t receive the reset email within a few minutes, check Spam/Promotions, then tell us the email you used and we’ll help. 

User: I’m not receiving the OTP / verification email.


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: Please check Spam/Promotions and confirm your mailbox isn’t full. Also ensure you’re using the correct email address. If it still doesn’t arrive, share your email and approximate time you tried—we can resend or verify your account manually. 

User: Thanks man


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



Assistant: (usually replies ‘Thank you’ or a kind message) I’m glad you could get help. If you have any more questions, feel free to ask. 

User: exit


In [ ]:

BASE_MODEL = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
ADAPTER_DIR = "./lora_adapter_out"
MERGED_DIR = "./merged_model"

# Load tokenizer (from adapter path)
tokenizer = AutoTokenizer.from_pretrained(ADAPTER_DIR)

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Load LoRA adapter
peft_model = PeftModel.from_pretrained(
    base_model,
    ADAPTER_DIR,
    torch_dtype=torch.float16
)

# Merge adapter weights into base model
merged_model = peft_model.merge_and_unload()

# Save merged model + tokenizer
merged_model.save_pretrained(MERGED_DIR)
tokenizer.save_pretrained(MERGED_DIR)

print("LoRA adapter successfully merged and saved to:", MERGED_DIR)